In [1]:
pip install huggingface rouge_score bert_score sacrebleu datasets transformers evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=67251c8fad667b222f26fea2b609fa59bc211dc447cef6b8be1da7571739fe7b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff85

In [2]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Define the data directory inside your Google Drive
# data_dir = "/content/drive/My Drive/Colab Notebooks/corpora"
data_dir = "/content/drive/My Drive/266 Data Project/corpora"

Mounted at /content/drive


In [ ]:
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import pandas as pd
import torch
# Load the pretrained MarianNMT model and tokenizer
def load_marian_model_and_tokenizer(model_name):
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

# Prepare dataset
def prepare_dataset(parquet_file, tokenizer, src_lang="chinese", tgt_lang="english"):
    # Load dataset from Parquet
    df = pd.read_parquet(parquet_file)

    # Tokenization function
    def preprocess_function(examples):
        model_inputs = tokenizer(
            examples[src_lang],
            max_length=128,
            truncation=True,
            padding="max_length",
        )
        labels = tokenizer(
            examples[tgt_lang],
            max_length=128,
            truncation=True,
            padding="max_length",
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    # Convert pandas DataFrame to Hugging Face Dataset
    hf_dataset = Dataset.from_pandas(df)
    tokenized_dataset = hf_dataset.map(preprocess_function, batched=True)
    return tokenized_dataset

# Train the model
def fine_tune_model(model, tokenizer, tokenized_dataset, output_dir):
    # Split dataset into train and validation
    dataset = tokenized_dataset.train_test_split(test_size=0.1)

    # Training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        save_strategy="epoch",
        logging_dir=f"{output_dir}/logs",
        logging_steps=100,
        predict_with_generate=True,
        push_to_hub=False,
    )

    # Trainer setup
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
    )

    # Train the model
    trainer.train()


In [ ]:
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import pandas as pd
import torch

# Main execution
if __name__ == "__main__":
    # Define model and file paths
    model_name = "Helsinki-NLP/opus-mt-en-zh"
    parquet_file = "/content/drive/MyDrive/266 Data Project/corpora/nejm/nejm_train.parquet"
    output_dir = "/content/drive/MyDrive/266 Data Project/corpora/nejm/fine_tuned_marian_model-en-zh"

    # Load the model and tokenizer
    tokenizer, model = load_marian_model_and_tokenizer(model_name)

    # Prepare the dataset
    tokenized_dataset = prepare_dataset(parquet_file, tokenizer, src_lang="english", tgt_lang="chinese")

    # Fine-tune the model
    fine_tune_model(model, tokenizer, tokenized_dataset, output_dir)


In [3]:
# Replace these file paths with your actual text files
# parquet_file = "/content/drive/MyDrive/Colab Notebooks/corpora/nejm/nejm_test.parquet"
# parquet_file = "/content/drive/MyDrive/266 Data Project/corpora/nejm/nejm_test.parquet"

# df = pd.read_parquet(
#     parquet_file)

# Translate Chinese sentences to English
# df["predicted_chinese"] = translate_sentences(tokenizer, model.to("cuda"), df["english"].tolist())
# df.to_parquet("/content/drive/MyDrive/Colab Notebooks/corpora/nejm/predictions/fine-tuned.parquet")
import pandas as pd
data = pd.read_parquet("/content/drive/MyDrive/266 Data Project/corpora/nejm/predictions/fine-tuned_en-zh.parquet")

In [4]:
data

,chinese,english,predicted_chinese
0,asciminib 是 与 BCR - ABL1 蛋白 的 豆蔻 酰 位点 相结合 的 别构...,asciminib is an allosteric inhibitor that bind...,"是 一种 , 可 结合 BCR - ABL1 的 , 通过 不同于 所有 其他 A..."
1,"asciminib 同时 靶向 作用 于 天然 和 突变 的 BCR - ABL1 , 包括...",asciminib targets both native and mutated BCR ...,"同时 和 BCR - ABL1 , 包括 T315I ."
2,asciminib 用于 费城 染色体 阳性 白血病 患者 的 安全性 和 抗 白血病 活性...,the safety and antileukemic activity of ascimi...,在 患者 中 的 和 抗 尚 不 清楚 .
3,"在 这项 1 期 剂量 递增 研究 中 , 我们 纳入 了 141 例 慢性期 和 9 例 ...","in this phase 1 , dose @-@ escalation study , ...","在 这项 1 期 剂量 研究 中 , 我们 纳入 了 141 例 慢性 患者 和 9 例..."
4,本 试验 的 主要 目的 是 确定 asciminib 的 最大 耐受 剂量 或 推荐 剂量...,the primary objective was to determine the max...,主要 目的 是 确定 的 最大 剂量 或 推荐 剂量 ( 或 两者 ) .
...,...,...,...
2097,"最近 对 患者 骨髓 标本 所 做 的 检测 显示 , 相同 的 基因 测序 获得 了 阴性...",recent testing of the patient 's bone marrow s...,"最近 对 患者 的 样本 进行 的 检测 发现 , 使用 相同 基因 的 结果 为 ."
2098,"因此 , 我们 认为 患者 发生 了 肿瘤 特异性 突变 .","therefore , the patient was believed to have a...","因此 , 患者 被 认为 有 肿瘤 ."
2099,目前 已经 发现 TNFAIP3 功能障碍 与 人类 自身 炎症性 疾病 以及 体外 刺激 ...,dysfunction of TNFAIP3 has been associated wit...,"在 刺激 过程 中 , TNFAIP3 与 人类 自身 疾病 相关 , 并且 与 T ..."
2100,解剖 诊断,anatomical Diagnosis,诊断


No charts were generated by quickchart


In [5]:
from evaluate import load

def evaluate_model_metrics(predictions, references, save_path=None):
    # Load the evaluation metrics
    bleu_metric = load("bleu")
    rouge_metric = load("rouge")
    bertscore_metric = load("bertscore")
    ter_metric = load("ter")

    # Format references for metric calculation
    references = [[ref] for ref in references]

    # Evaluate BLEU score
    bleu_result = bleu_metric.compute(predictions=predictions, references=references)

    # Evaluate ROUGE score
    rouge_result = rouge_metric.compute(predictions=predictions, references=references)

    # Evaluate BERTScore
    bertscore_result = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

    # Evaluate TER (Translation Edit Rate)
    ter_result = ter_metric.compute(predictions=predictions, references=references)

    # Extract summary statistics for BERTScore
    bertscore_summary = {
        "mean": sum(bertscore_result["f1"]) / len(bertscore_result["f1"]),
        "median": sorted(bertscore_result["f1"])[len(bertscore_result["f1"]) // 2],
        "std": (sum((x - sum(bertscore_result["f1"]) / len(bertscore_result["f1"]))**2 for x in bertscore_result["f1"]) / len(bertscore_result["f1"]))**0.5
    }

    # Consolidate results
    results = {
        "BLEU": bleu_result,
        "ROUGE": rouge_result,
        "BERTScore": bertscore_summary,
        "TER": ter_result,
    }

    return results

In [6]:
truth = data['chinese'].to_list()
predictions = data['predicted_chinese'].to_list()
evaluate_model_metrics(predictions, truth)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'BLEU': {'bleu': 0.3291534676419385,
  'precisions': [0.7695794462834787,
   0.4780590965164777,
   0.32609938344698053,
   0.2347989173291277],
  'brevity_penalty': 0.803439457047837,
  'length_ratio': 0.8204431825195976,
  'translation_length': 53168,
  'reference_length': 64804},
 'ROUGE': {'rouge1': 0.6222200352516298,
  'rouge2': 0.4095451302645647,
  'rougeL': 0.6106616902520141,
  'rougeLsum': 0.6109070226858001},
 'BERTScore': {'mean': 0.9223777424653976,
  'median': 0.9234125018119812,
  'std': 0.04260976834518433},
 'TER': {'score': 46.578332034294625,
  'num_edits': 29880,
  'ref_length': 64150.0}}